In [1]:
import pandas as pd
import numpy as np
import datetime
import xlrd
%config Completer.use_jedi = False


In [97]:
stk_off = pd.DataFrame({'Ticker': ['EWZ US 05/29/20 P27', 'HSI US 12/30/20 C24800', 'TLT US 06/19/20 C225', 'EWZ US 05/29/20 P29', 'XLE US 05/29/20 P30'],
                   'Instrument': ['EWZ', 'HSI', 'TLT', 'EWZ', 'EWZ'],
                   'Maturity Label': ['MAI 20 D29', 'DEC 20 D30', 'JUN 20 D19', 'MAY 20 D29', 'MAY 20 D29'],
                   'Strike': [27, 24800, 225, 29, 29],
                   'Payout': ['P', 'C', 'C', 'P', 'P'],
                   'Style': ['A', 'E', 'E', 'A', 'A'],
                   'Maturity': ['29/05/2020', '12/30/2020', '19/06/2020', '29/05/2020', '29/05/2020'],
                   'Market': ['US NYSE', 'US NYSE', 'HK HKSE', 'US NYSE', 'US NYSE']})

stk_off_original = stk_off.copy()

stk_off['Reduced_Ticker'] = stk_off['Ticker'].apply(lambda a :" ".join(a.split(" ", 2)[:2]))

In [44]:
def organize_df(df):
    df = df[['Ticker', 'Instrument', 'Maturity Label', 'Strike', 'Payout', 'Style', 'Maturity', 'Market']]
    return df

In [3]:
def market_ticker(stk_off):
    market_ticker_var = stk_off[['Market','Reduced_Ticker']].groupby(['Market']).agg(list)
    market_ticker_var['Reduced_Ticker'] = market_ticker_var['Reduced_Ticker'].apply(lambda x: pd.unique(x))
    market_ticker_var = market_ticker_var.explode('Reduced_Ticker')
    market_ticker_var = market_ticker_var.reset_index()
    
    return market_ticker_var

In [4]:
def market_column(market_ticker, tickers, df):
    tickers_reduced = [t.split()[0] for t in tickers]
    
    selection_market = (market_ticker
                   .explode('Reduced_Ticker')
                   ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))


    market = market_ticker.loc[selection_market].Market.to_list()
        
    df['Market'] = market
    
    return df

In [5]:
def instrument_ticker(stk_off):
    instrument_ticker_var = stk_off[['Instrument','Reduced_Ticker']].groupby(['Instrument']).agg(list)
    instrument_ticker_var['Reduced_Ticker'] = instrument_ticker_var['Reduced_Ticker'].apply(lambda x: pd.unique(x))
    instrument_ticker_var = instrument_ticker_var.explode('Reduced_Ticker')
    instrument_ticker_var = instrument_ticker_var.reset_index()
    
    return instrument_ticker_var

In [6]:
def instrument_column(instrument_ticker, tickers, df):
    tickers_reduced = [t.split()[0] for t in tickers]
        
    selection_instrument = (instrument_ticker
             .explode('Reduced_Ticker')
             ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))

    instrument = instrument_ticker.loc[selection_instrument].Instrument.to_list()
     
    df['Instrument'] = instrument

    return df

In [7]:
def style(df):
    df['Style'] = ''
    for x in df['Market']:
        if x == 'US NYSE':
            df['Style'] = 'A'
        else:
            df['Style'] = 'E'
    
    return df

In [8]:
def payout_strike(df, tickers):
    payout_strike = [t.split()[3] for t in tickers]
    df['Payout'] = [p[0] for p in payout_strike]
    df['Strike'] = [s[1:] for s in payout_strike]
    
    return df

In [55]:
def maturity(df):
    df['Maturity'] = [datetime.datetime.strptime(x.split()[2], '%m/%d/%y').strftime('%d/%m/%Y') for x in df['Ticker']]
    return df

In [10]:
def read_maturities():
    maturities_usnyse = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Vencimentos/US NYSE.xlsx')
    maturities_usnyse['Maturity'] = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y') 
                     for x in maturities_usnyse.Maturity]
    return maturities_usnyse

In [11]:
read_maturities()

/home/gustavo/Desktop/Fut_Opt/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,Label,First Td,Maturity
0,MAI 20 D29,NaN,29/05/2020
1,DEC 20 D30,NaN,30/12/2020


In [23]:
def maturity_label(df):
    maturities_usnyse = read_maturities()
    selection_label = (maturities_usnyse.explode('Maturity')['Maturity'].apply(lambda x: x in df['Maturity'].to_list()))
                                                                                                
    df['Maturity Label'] = maturities_usnyse.loc[selection_label].Label.to_list()
    return df

In [25]:
tickers = ['EWZ US 05/29/20 C31','HSI US 12/30/20 C24900']
df_to_upload = pd.DataFrame()
df_to_upload['Tickers'] = tickers

market_ticker_var = market_ticker(stk_off)
df_to_upload = market_column(market_ticker_var, tickers, df_to_upload)

instrument_ticker_var = instrument_ticker(stk_off)
df_to_upload = instrument_column(instrument_ticker_var, tickers, df_to_upload)

df_to_upload = style(df_to_upload)

df_to_upload = payout_strike(df_to_upload, tickers)

df_to_upload = maturity(df_to_upload)
df_to_upload = maturity_label(df_to_upload)

df_to_upload

/home/gustavo/Desktop/Fut_Opt/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,Tickers,Market,Instrument,Style,Payout,Strike,Maturity,Maturity Label
0,EWZ US 05/29/20 C31,US NYSE,EWZ,A,C,31,29/05/2020,MAI 20 D29
1,HSI US 12/30/20 C24900,US NYSE,HSI,A,C,24900,30/12/2020,DEC 20 D30


In [105]:
def file_to_upload(tickers, base, original_base):
    for t in tickers:
        if original_base['Ticker'].str.contains(t).any():
            tickers.remove(t)
            print('Ticker ' + t + 'já consta na base')
    
    if not tickers:
        return "Não há tickers a serem cadastrados"
    
    df_to_upload = pd.DataFrame()
    df_to_upload['Ticker'] = tickers

    market_ticker_var = market_ticker(stk_off)
    df_to_upload = market_column(market_ticker_var, tickers, df_to_upload)

    instrument_ticker_var = instrument_ticker(stk_off)
    df_to_upload = instrument_column(instrument_ticker_var, tickers, df_to_upload)

    df_to_upload = style(df_to_upload)

    df_to_upload = payout_strike(df_to_upload, tickers)

    df_to_upload = maturity(df_to_upload)
    df_to_upload = maturity_label(df_to_upload)
    df_to_upload = organize_df(df_to_upload)
    
    original_base = original_base.append(df_to_upload, ignore_index=True)
    #df_to_upload.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Arquivo_para_upload.xlsx')

    return df_to_upload, original_base

In [108]:
x, y = file_to_upload(tickers, stk_off, stk_off_original)
x

/home/gustavo/Desktop/Fut_Opt/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,Ticker,Instrument,Maturity Label,Strike,Payout,Style,Maturity,Market
0,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,A,29/05/2020,US NYSE
1,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,A,30/12/2020,US NYSE


In [109]:
y

,Ticker,Instrument,Maturity Label,Strike,Payout,Style,Maturity,Market
0,EWZ US 05/29/20 P27,EWZ,MAI 20 D29,27,P,A,29/05/2020,US NYSE
1,HSI US 12/30/20 C24800,HSI,DEC 20 D30,24800,C,E,12/30/2020,US NYSE
2,TLT US 06/19/20 C225,TLT,JUN 20 D19,225,C,E,19/06/2020,HK HKSE
3,EWZ US 05/29/20 P29,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
4,XLE US 05/29/20 P30,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
5,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,A,29/05/2020,US NYSE
6,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,A,30/12/2020,US NYSE


In [110]:
for t in tickers:
    if y['Ticker'].contains(t).any():
        tickers.remove(t)
        print('Ticker ' + t + 'já consta na base')
    else:
        print(t)

AttributeError: 'Series' object has no attribute 'contains'

In [93]:
stk_off_original.append(x, ignore_index=True)

,Ticker,Instrument,Maturity Label,Strike,Payout,Style,Maturity,Market
0,EWZ US 05/29/20 P27,EWZ,MAI 20 D29,27,P,A,29/05/2020,US NYSE
1,HSI US 12/30/20 C24800,HSI,DEC 20 D30,24800,C,E,12/30/2020,US NYSE
2,TLT US 06/19/20 C225,TLT,JUN 20 D19,225,C,E,19/06/2020,HK HKSE
3,EWZ US 05/29/20 P29,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
4,XLE US 05/29/20 P30,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
5,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,A,29/05/2020,US NYSE
6,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,A,30/12/2020,US NYSE


In [95]:
stk_off_original = stk_off_original.append(x)

In [96]:
stk_off_original

,Ticker,Instrument,Maturity Label,Strike,Payout,Style,Maturity,Market
0,EWZ US 05/29/20 P27,EWZ,MAI 20 D29,27,P,A,29/05/2020,US NYSE
1,HSI US 12/30/20 C24800,HSI,DEC 20 D30,24800,C,E,12/30/2020,US NYSE
2,TLT US 06/19/20 C225,TLT,JUN 20 D19,225,C,E,19/06/2020,HK HKSE
3,EWZ US 05/29/20 P29,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
4,XLE US 05/29/20 P30,EWZ,MAY 20 D29,29,P,A,29/05/2020,US NYSE
0,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,A,29/05/2020,US NYSE
1,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,A,30/12/2020,US NYSE
